Titanic


In [1]:
import pandas as pd

df = pd.read_csv('../data/01_raw/train.csv')
display(df.head())


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
import pandas as pd

df2 = pd.read_csv('../data/01_raw/test.csv')
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
df_all = pd.concat([df,df2])
df_all.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200



## Find and handle nulls

In [4]:
df_all = pd.concat([df,df2])

ss2 = df_all.isnull().count()
ct2 = df_all.isnull().sum()
dfn = pd.DataFrame([ss2,ct2, 100*ct2/ss2]).T
dfn.columns=['Total','Missing','Pct']
dfn = dfn.sort_values(by='Pct', ascending=False)
display(dfn)



,Total,Missing,Pct
Cabin,1309.0,1014.0,77.463713
Survived,1309.0,418.0,31.932773
Age,1309.0,263.0,20.091673
Embarked,1309.0,2.0,0.152788
Fare,1309.0,1.0,0.076394
PassengerId,1309.0,0.0,0.000000
Pclass,1309.0,0.0,0.000000
Name,1309.0,0.0,0.000000
Sex,1309.0,0.0,0.000000
SibSp,1309.0,0.0,0.000000


## Strategies
- Cabin -- too many blanks and weird string data, maybe drop this.  or convert to A/B/C ... 0
- Age -- pretty important, set mean/mode as whole or by gender, pclass, etc
- Embarked -- port of embark...  see if there is any correlation of class, gender, etc to set default
- Fare is the price for each seat?  mean or mean based on class ?


In [5]:
# Age
import numpy as np

display(df_all[['Pclass','Sex','Age']].groupby(['Sex','Pclass']).mean())  # build groupings

df_all2 = df_all.copy()
df_all2['Age'].fillna(df_all2[['Pclass','Sex','Age']].groupby(['Sex','Pclass']).transform(np.mean).iloc[:,0], inplace=True)
display(df_all2.tail())

df_all2['QBinAge'] = pd.qcut(df_all2['Age'], q=5, labels=[1,2,3,4,5])
df_all2

Age
Sex    Pclass           
female 1       37.037594
       2       27.499223
       3       22.185329
male   1       41.029272
       2       30.815380
       3       25.962264

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,NaN,3,"Spector, Mr. Woolf",male,25.962264,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,25.962264,0,0,359309,8.0500,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,25.962264,1,1,2668,22.3583,NaN,C


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,QBinAge
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,4
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,3
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,4
4,5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,25.962264,0,0,A.5. 3236,8.0500,NaN,S,2
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,4
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,4
416,1308,NaN,3,"Ware, Mr. Frederick",male,25.962264,0,0,359309,8.0500,NaN,S,2


In [6]:
# Fare avg ?
df_all[df_all['Fare'].isnull()]
df_all['Fare'].isnull().sum()
df_all[['Fare','Pclass']].groupby(['Pclass']).mean()
# display(df_all[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).mean())  # build groupings
# display(df_all[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).transform(np.mean))  # apply to each row if null
# display(df_all['Fare'])
df_all2['Fare'].fillna(df_all2[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).
                       transform(np.mean).iloc[:,0], inplace=True)
df_all2.tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,QBinAge
413,1305,NaN,3,"Spector, Mr. Woolf",male,25.962264,0,0,A.5. 3236,8.0500,NaN,S,2
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,4
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,4
416,1308,NaN,3,"Ware, Mr. Frederick",male,25.962264,0,0,359309,8.0500,NaN,S,2
417,1309,NaN,3,"Peter, Master. Michael J",male,25.962264,1,1,2668,22.3583,NaN,C,2


In [7]:
# example of multi-level index joins
index_left = pd.MultiIndex.from_tuples([('K0', 'X0'), ('K0', 'X1'),('K1', 'X2')],names=['key', 'X'])

left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],'B': ['B0', 'B1', 'B2']}, index=index_left)
display(left)

index_right = pd.MultiIndex.from_tuples([('K0', 'X0'), ('K0', 'X1'),('K2', 'X2'), ('K2', 'X3')], names=['key', 'X'])
right = pd.Series(['C0', 'C1', 'C2','C3'], index=index_right, name='right')
display(right.reset_index())

left.join(right)


A   B
key X         
K0  X0  A0  B0
    X1  A1  B1
K1  X2  A2  B2

,key,X,right
0,K0,X0,C0
1,K0,X1,C1
2,K2,X2,C2
3,K2,X3,C3


A   B right
key X               
K0  X0  A0  B0    C0
    X1  A1  B1    C1
K1  X2  A2  B2   NaN

In [12]:
# Port Embarked -> match to most frequent match by class,sex and port of others ?
# display(df_all[['Pclass','Sex','Embarked','QBinAge','Fare']].groupby(['Sex','Pclass','QBinAge','Embarked']).count())
portmap = df_all2[['Pclass','Sex','Embarked','QBinAge','Fare']].groupby(['Sex','Pclass','QBinAge']).apply(
         lambda x: x.sort_values('Fare').head(1)['Embarked'].values[0])

portmap = portmap.rename('NewEmb').reset_index()
#display(portmap.head())

df_all2 = pd.merge(how='inner',left=df_all2, right=portmap)
display(df_all2[df_all2.Embarked.isnull()])
# alot of work for filling 2 null values...
df_all3 = df_all2.copy()
df_all3['Embarked'] = df_all2['Embarked'].fillna(df_all2['NewEmb'])

display(df_all3.iloc[207,:].reset_index().T)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,QBinAge,NewEmb
207,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,4,C
693,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,5,S


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,QBinAge,NewEmb
207,62,1,1,"Icard, Miss. Amelie",female,38,0,0,113572,80,B28,C,4,C



# Gameplan 1.0

- Assumptions
  - Low class got stuck (as per movie)
  - Higher class folks got out first
  - Older folks may not have made it?
  - Preference to children and famlies 
  
- Likely columns:
  - Class, Sex, Age, Sibling#, Parent/Child#, Fare, Cabin 

- Maybe columns:
  - Ticket - any patern of fare/class to ticket# ? (corr to fare/class)
  - Embarked - some ports poor or worse cabins?    (corr to fare/class)

Steps
1.  Find correlations between features
2.  OneHot Encode features
3.  Run initial logistic model +/- features 

b

In [ ]:
df_all['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0].unique()


# Target Submission format

In [ ]:
df = pd.read_csv('../data/01_raw/gender_submission.csv')
df.head()
